### Raw event example analysis

Be sure to run the first three sections in the  [retrosheet-spark](retrosheet-spark.ipynb) notebook through the sectiatuon that loads "EVENT FILES".

In [3]:
## this is an example which looks at game duration info records

def filterRecsWithDuration(rec):
    try:
        event = rec[4]
        return event[0] == 'info' and event[1] == 'timeofgame' and int(event[2])>0
    except:
        return False

def makeLocationDurationRow(rec):
    
    gameDate=rec[3]
    home=rec[2]
    duration=int(rec[4][2])
    return Row(gameDate=gameDate,home=home,duration=duration)    
    
datesTeamsDurationDF = combinedFlattenedEvents.filter(filterRecsWithDuration).map(makeLocationDurationRow).toDF()
yearsTeamsDurationDF = datesTeamsDurationDF.withColumn("gameYear", year("gameDate"))
yearsTeamsDurationDF.show()

yearsTeamsDurationDF.write.format("parquet").save(YEAR_TEAMS_OUTPUT_PATH)
print("done!")

NameError: name 'combinedFlattenedEvents' is not defined

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

df = spark.read.load(YEAR_TEAMS_OUTPUT_PATH)
homeGameYearDF = df.groupBy("gameYear", "home").agg(avg("duration").alias("avgDuration"))

data=[]

for row in homeGameYearDF.select("home").distinct().collect():
    
    homeDf = homeGameYearDF.filter(homeGameYearDF.home==row.home).orderBy("gameYear").toPandas()

    data.append(
        go.Scatter(x=homeDf.gameYear, y=homeDf.avgDuration, name=row.home)
    )
    
plotly.offline.iplot(data)



NameError: name 'YEAR_TEAMS_OUTPUT_PATH' is not defined